<a href="https://colab.research.google.com/github/CJKang0601/GenAI_Course_NCCU/blob/main/GAI_final_project_ipyb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IELTS 雅思BRO，讓你YES


In [ ]:
# ===================================================================
# 1. 環境設定與安裝
# ===================================================================
!pip install spacy google-generativeai -q
!python -m spacy download en_core_web_lg -q

# 載入套件
import spacy
import google.generativeai as genai
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
import json
import random
from google.colab import userdata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 1.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
# ===================================================================
# 2. API 與模型設定
# ===================================================================

try:
    # 根據您的設定，使用 'GOOGLE_API_KEY' 作為金鑰名稱
    API_KEY = userdata.get('GOOGLE_API_KEY')
    genai.configure(api_key=API_KEY)
    model = genai.GenerativeModel('gemini-1.5-flash')
    print("Gemini API 設定成功！")
except Exception as e:
    print(f"無法設定 Gemini API，請檢查您的 API 金鑰。錯誤：{e}")
    model = None

print("正在載入 NLP 模型...")
nlp = spacy.load("en_core_web_lg")
print("模型載入完成！")

Gemini API 設定成功！
正在載入 NLP 模型...
模型載入完成！


In [ ]:
# ===================================================================
# 3. 生成式 AI 核心函式
# ===================================================================

def generate_reading_content(topic):
    """使用 Gemini API 動態生成閱讀文章、題目和答案"""
    if not model: return None, "API is not configured."
    generation_config = genai.GenerationConfig(temperature=0.8)
    prompt = f"""
    You are an expert IELTS exam creator. Generate a short academic passage (150-200 words) on the topic of '{topic}' and a set of 3 related questions: 1 MCQ, 1 TFNG, and 1 Sentence Completion.
    Output MUST be in this exact JSON format:
    {{
      "passage_title": "...", "passage_text": "...",
      "questions": [
        {{"question": "...", "options": ["A) ...", "B) ...", "C) ..."], "answer": "...", "type": "mcq"}},
        {{"question": "...", "answer": "...", "type": "tfng"}},
        {{"question": "...", "answer": "...", "type": "sentence_completion"}}
      ]
    }}
    """
    try:
        response = model.generate_content(prompt, generation_config=generation_config)
        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
        return json.loads(cleaned_response), None
    except Exception as e:
        return None, f"生成內容時發生錯誤: {e}"

# --- 本次新增函式 ---
def generate_writing_task_with_llm(topic):
    """使用 Gemini API 動態生成雅思寫作 Task 2 題目"""
    if not model: return None, "API is not configured."
    generation_config = genai.GenerationConfig(temperature=0.9) # 題目可以更有創意
    prompt = f"""
    You are an expert IELTS exam creator. Your task is to generate one IELTS Writing Task 2 question on the topic of '{topic}'.
    The question should be one of the common IELTS formats, such as 'To what extent do you agree or disagree?', 'Discuss both views and give your own opinion.', or 'What are the causes of this problem and what are some potential solutions?'.

    Please provide the output STRICTLY in the following JSON format:
    {{
        "task_text": "The full text of the IELTS Writing Task 2 question.",
        "task_category": "{topic}"
    }}
    """
    try:
        response = model.generate_content(prompt, generation_config=generation_config)
        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
        return json.loads(cleaned_response), None
    except Exception as e:
        return None, f"生成寫作題目時發生錯誤: {e}"
# --- 新增結束 ---

def evaluate_writing_with_llm(writing_task, essay):
    """使用 Gemini API 根據雅思標準評估作文"""
    if not model: return None, "API is not configured."
    generation_config = genai.GenerationConfig(temperature=0.7)
    prompt = f"""
    You are an experienced IELTS writing examiner. Evaluate the essay based on the provided task and official criteria (TA, CC, LR, GRA).
    **The Writing Task:** {writing_task}
    **The Student's Essay:** {essay}
    Provide a detailed evaluation in a STRICT JSON format with scores for each criterion (4.0-9.0), feedback, an overall score, and a summary.
    **JSON Output Format:**
    {{
      "scores": {{
        "task_achievement": {{"score": 0.0, "feedback": "..."}}, "coherence_cohesion": {{"score": 0.0, "feedback": "..."}},
        "lexical_resource": {{"score": 0.0, "feedback": "..."}}, "grammatical_range_accuracy": {{"score": 0.0, "feedback": "..."}}
      }}, "overall_score": 0.0, "summary_feedback": "..."
    }}
    """
    try:
        response = model.generate_content(prompt, generation_config=generation_config)
        cleaned_response = response.text.strip().replace("```json", "").replace("```", "")
        return json.loads(cleaned_response), None
    except Exception as e:
        return None, f"分析作文時發生錯誤: {e}"

def calculate_reading_score(correct_count, total_questions):
    if total_questions == 0: return 0.0
    ratio = correct_count / total_questions
    if ratio == 1.0: return 9.0
    elif ratio >= 0.8: return 8.0
    elif ratio >= 0.66: return 7.0
    elif ratio >= 0.5: return 6.0
    elif ratio >= 0.33: return 5.0
    else: return 4.0


In [ ]:
# ===================================================================
# 4. UI 介面元件定義 (寫作區元件已修改)
# ===================================================================

topic_selector = widgets.Dropdown(options=["Technology", "Environment", "Health & Medicine", "Education", "Society"], description='選擇主題:', style={'description_width': 'initial'})

# --- 閱讀區 ---
generate_reading_btn = widgets.Button(description="生成新的閱讀測驗", button_style='primary')
reading_content_box = widgets.VBox([])
reading_submit_btn = widgets.Button(description="提交閱讀答案", button_style='success')
reading_results_output = widgets.Output()

# --- 寫作區 ---
generate_writing_btn = widgets.Button(description="生成新的寫作題目", button_style='primary')
writing_task_output = widgets.Output() # 從靜態 HTML 改為可動態更新的 Output
writing_input = widgets.Textarea(placeholder="請在此輸入你的作文... (建議至少250字)", layout=widgets.Layout(height='300px', width='95%'))
writing_submit_btn = widgets.Button(description="使用 AI 進行分析", button_style='info')
writing_feedback_output = widgets.Output()

# 全局變數，用於儲存當前生成的題目和答案框
current_reading_data = {}
current_answer_widgets = []
current_writing_task = "" # 用於儲存當前的寫作題目


In [ ]:
# ===================================================================
# 5. 事件處理函式
# ===================================================================

def on_generate_reading(b):
    global current_reading_data, current_answer_widgets
    reading_content_box.children = []
    reading_results_output.clear_output()
    loading_html = widgets.HTML("<h4><br>正在由 AI 生成全新的文章和題目，請稍候...</h4>")
    reading_content_box.children = [loading_html]
    topic = topic_selector.value
    generated_data, error = generate_reading_content(topic)
    if error:
        reading_content_box.children = [widgets.HTML(f"<p style='color:red;'>{error}</p>")]
        return
    current_reading_data = generated_data
    current_answer_widgets = []
    question_type_map = {"mcq": "選擇題 (Multiple Choice)", "tfng": "是非/未提及題 (True/False/Not Given)", "sentence_completion": "句子填空 (Sentence Completion)"}
    passage_set = current_reading_data
    content_widgets = []
    passage_html = f"<h3>{passage_set['passage_title']}</h3><div>{passage_set['passage_text']}</div><hr>"
    content_widgets.append(widgets.HTML(passage_html))
    questions_vbox_children = []
    for q_data in passage_set['questions']:
        question_type_full_name = question_type_map.get(q_data['type'], q_data['type'].replace('_', ' ').title())
        type_label = widgets.HTML(f"<p style='background-color:#E0E0E0; color:#333; padding:2px 10px; border-radius:12px; font-size:0.9em; display:inline-block; margin-bottom: -5px;'>{question_type_full_name}</p>")
        question_text = q_data["question"]
        if q_data['type'] == 'mcq':
            options_text = "<br>" + "<br>".join(q_data['options'])
            question_text += options_text
        question_label = widgets.HTML(value=f"<p>{question_text}</p>")
        answer_input = widgets.Text(placeholder="請在此輸入答案")
        current_answer_widgets.append(answer_input)
        questions_vbox_children.append(widgets.VBox([type_label, question_label, answer_input]))
    content_widgets.append(widgets.VBox(questions_vbox_children))
    reading_content_box.children = tuple(content_widgets)

# --- 事件處理 ---
def on_generate_writing(b):
    global current_writing_task
    writing_task_output.clear_output()
    writing_feedback_output.clear_output() # 清空上次的評分
    writing_input.value = "" # 清空上次的作文

    with writing_task_output:
        display(HTML("<h4>正在由 AI 生成全新的寫作題目，請稍候...</h4>"))
        topic = topic_selector.value
        task_data, error = generate_writing_task_with_llm(topic)
        clear_output(wait=True) # 清除"生成中"的訊息

        if error:
            display(HTML(f"<p style='color:red;'>{error}</p>"))
            current_writing_task = ""
            return

        # 更新全域變數和UI
        current_writing_task = task_data['task_text']
        display(HTML(f"<h4>寫作題目 ({task_data['task_category']})</h4><p>{current_writing_task}</p>"))
# --- 新增結束 ---

def on_reading_submit(b):
    # ... (此函式不變) ...
    global current_reading_data, current_answer_widgets
    if not current_reading_data:
        reading_results_output.clear_output()
        with reading_results_output: display(HTML("<p style='color:orange;'>請先生成閱讀測驗。</p>"))
        return
    reading_results_output.clear_output()
    all_correct_answers = [q['answer'] for q in current_reading_data['questions']]
    total_questions = len(all_correct_answers)
    correct_count = 0
    for i, correct_answer in enumerate(all_correct_answers):
        user_answer = current_answer_widgets[i].value.strip()
        if user_answer.lower() == correct_answer.lower(): correct_count += 1
    final_score = calculate_reading_score(correct_count, total_questions)
    with reading_results_output:
        display(HTML(f"""<div style='border: 2px solid #4CAF50; padding: 10px; border-radius: 5px; margin-top: 15px;'><h3>閱讀測驗結果</h3><p>答對題數：<b>{correct_count} / {total_questions}</b></p><p>預估雅思分數：<b style='font-size: 1.5em; color: #D32F2F;'>{final_score}</b></p></div>"""))

def on_writing_submit(b):
    # --- 從全域變數讀取題目 ---
    global current_writing_task
    if not current_writing_task:
        writing_feedback_output.clear_output()
        with writing_feedback_output:
            display(HTML("<p style='color:orange;'>請先生成寫作題目。</p>"))
        return

    writing_feedback_output.clear_output()
    with writing_feedback_output:
        user_essay = writing_input.value
        if len(user_essay.split()) < 50:
            display(HTML("<p style='color: red;'>錯誤：作文內容太短，AI 無法進行有效分析。請至少輸入50個單字。</p>"))
            return
        display(HTML("<h4>AI 正在深度分析您的作文，這可能需要一點時間...</h4>"))
        evaluation, error = evaluate_writing_with_llm(current_writing_task, user_essay)
        clear_output(wait=True)
        if error:
            display(HTML(f"<p style='color:red;'>{error}</p>"))
            return
        scores = evaluation['scores']
        feedback_html = f"""<div style='border: 2px solid #1976D2; padding: 15px; border-radius: 5px;'><h2>AI 寫作分析報告</h2><h3>總分: <span style='font-size: 1.5em; color: #D32F2F;'>{evaluation['overall_score']}</span></h3><p><b>總體評價:</b> {evaluation['summary_feedback']}</p><hr><h4>各項指標分析:</h4><ul style='list-style-type: none; padding-left: 0;'><li><b>任務完成度 (TA): {scores['task_achievement']['score']}</b><p style='margin-left: 20px; font-style: italic;'>"{scores['task_achievement']['feedback']}"</p></li><li><b>連貫與銜接 (CC): {scores['coherence_cohesion']['score']}</b><p style='margin-left: 20px; font-style: italic;'>"{scores['coherence_cohesion']['feedback']}"</p></li><li><b>詞彙資源 (LR): {scores['lexical_resource']['score']}</b><p style='margin-left: 20px; font-style: italic;'>"{scores['lexical_resource']['feedback']}"</p></li><li><b>語法廣度與準確性 (GRA): {scores['grammatical_range_accuracy']['score']}</b><p style='margin-left: 20px; font-style: italic;'>"{scores['grammatical_range_accuracy']['feedback']}"</p></li></ul></div>"""
        display(HTML(feedback_html))

In [ ]:
# ===================================================================
# 6. 最終介面佈局與顯示
# ===================================================================
generate_reading_btn.on_click(on_generate_reading)
generate_writing_btn.on_click(on_generate_writing) # 綁定新按鈕
reading_submit_btn.on_click(on_reading_submit)
writing_submit_btn.on_click(on_writing_submit)

# 這邊將主題選擇器和生成按鈕放在一起
common_controls = widgets.HBox([
    topic_selector,
    generate_reading_btn,
    generate_writing_btn
])

reading_section = widgets.VBox([reading_content_box, reading_submit_btn, reading_results_output])
writing_section = widgets.VBox([writing_task_output, writing_input, writing_submit_btn, writing_feedback_output])
app = widgets.Tab(children=[reading_section, writing_section])
app.set_title(0, 'AI 生成閱讀練習')
app.set_title(1, 'AI 生成寫作練習')

# 最終顯示
display(common_controls, app)

# 首次打開時會自動生成一篇閱讀和一篇寫作題目
on_generate_reading(None)
on_generate_writing(None)